In [ ]:
# Import Modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [157]:
df = pd.read_csv(
    Path('updated_player_data_VV.csv')
)
df.head()

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,PreferredFoot,...,Skill,Movement,Power,Mentality,Defending,Goalkeeping,AllStats,Days Joined,AttackingWorkRate,DefensiveWorkRate
0,158023,L. Messi,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,0,...,467.0,458.0,378.0,429.0,87.0,54.0,2298.0,5819.0,1,1
1,20801,Cristiano Ronaldo,33,Portugal,94,94,Juventus,77000000.0,405000.0,1,...,416.0,433.0,450.0,449.0,82.0,58.0,2323.0,697.0,2,0
2,190871,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,1,...,444.0,458.0,353.0,443.0,84.0,59.0,2237.0,1038.0,2,1
3,193080,De Gea,27,Spain,91,93,Manchester United,72000000.0,260000.0,1,...,151.0,308.0,217.0,256.0,49.0,444.0,1539.0,3263.0,1,1
4,192985,K. De Bruyne,27,Belgium,91,92,Manchester City,102000000.0,355000.0,1,...,436.0,401.0,410.0,485.0,177.0,56.0,2369.0,1742.0,2,2


In [158]:
# Clean and convert Wage column
df['Wage'] = df['Wage'].replace('[\€,K]', '', regex=True)  
df['Wage'] = pd.to_numeric(df['Wage'], errors='coerce')   

In [159]:
# Check for missing values
print("Missing values per column:\n", df.isnull().sum())

Missing values per column:
 ID                      0
Name                    0
Age                     0
Nationality             0
Overall                 0
                     ... 
Goalkeeping             0
AllStats                0
Days Joined          1264
AttackingWorkRate       0
DefensiveWorkRate       0
Length: 64, dtype: int64


In [ ]:
#Preparing the data
df2 = df.drop(['ID','DaysJoined','Value','Potential'],axis=1)
df2 = df2[df2['Position'] != 'GK']
df2 = df2.select_dtypes(include=['number'])
df2.head()

,Age,Overall,Wage,PreferredFoot,InternationalReputation,WeakFoot,SkillMoves,Height,Weight,Crossing,...,Attacking,Skill,Movement,Power,Mentality,Defending,Goalkeeping,AllStats,AttackingWorkRate,DefensiveWorkRate
0,31,94,565000.0,0,5.0,4.0,4.0,67.0,159.0,84.0,...,425.0,467.0,458.0,378.0,429.0,87.0,54.0,2298.0,1,1
1,33,94,405000.0,1,5.0,4.0,5.0,74.0,183.0,84.0,...,435.0,416.0,433.0,450.0,449.0,82.0,58.0,2323.0,2,0
2,26,92,290000.0,1,5.0,5.0,5.0,69.0,150.0,79.0,...,396.0,444.0,458.0,353.0,443.0,84.0,59.0,2237.0,2,1
4,27,91,355000.0,1,4.0,5.0,4.0,71.0,154.0,93.0,...,404.0,436.0,401.0,410.0,485.0,177.0,56.0,2369.0,2,2
5,27,91,340000.0,1,4.0,4.0,4.0,68.0,163.0,81.0,...,395.0,434.0,461.0,367.0,448.0,83.0,45.0,2233.0,2,1


In [161]:
# Splitting the data
y = df2['Wage']
X = df2.drop(columns='Wage')
# Split data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=38)

In [162]:
# Scaling the data
# Create the StandardScaler instance
scaler = StandardScaler()
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Linear Regression

In [163]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [164]:
print("Model Coefficients:", model.coef_)
print("Model Intercept:", model.intercept_)

Model Coefficients: [-3.45419894e+03  9.48627706e+03  1.33112107e+02  1.11808097e+04
  1.03476460e+02 -3.86040494e+02  6.14957625e+02  2.94848828e+02
 -2.58743152e+14 -2.99173991e+14 -2.11492598e+14 -1.78383866e+14
 -2.69852992e+14 -6.70159289e+14 -8.12282649e+14 -8.05451649e+14
 -6.62222235e+14 -5.36405769e+14  5.30394190e+13  5.19621043e+13
  5.57715290e+13  3.95532791e+13  5.50861669e+13  1.58515244e+15
  1.38804780e+15  1.34199388e+15  1.50536038e+15  1.89006082e+15
 -5.89974582e+14 -7.68896448e+14 -6.01639581e+14 -5.28086115e+14
 -5.11120928e+14 -4.13583276e+14 -1.32084878e+14 -1.47672027e+14
 -1.47690147e+14 -2.41138944e+13 -2.39072276e+13 -2.44100803e+13
 -2.39451778e+13 -2.40627901e+13 -5.27818560e+14  1.32680768e+15
 -1.49690881e+15 -5.95577409e+15  5.60891887e+14 -1.17435407e+15
 -1.72998285e+14  6.05499171e+15  4.65210938e+02  8.41998047e+02]
Model Intercept: 10178.907263617513


In [165]:
# Predict on the test data
y_pred = model.predict(X_test_scaled)
# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)

# Compare a few predictions vs actual values
comparison_df = pd.DataFrame({'Actual': y_test.values, 'Predicted': y_pred})
print(comparison_df.head())

Mean Squared Error (MSE): 291159517.5273114
R-squared (R2): 0.533731777119838
   Actual    Predicted
0  3000.0  4288.831712
1  6000.0  4212.375316
2  1000.0  -264.336023
3  7000.0  7886.413977
4  2000.0 -1637.336023


# Random Forest

In [166]:
# Initialize and train Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Predict and evaluate
rf_pred = rf_model.predict(X_test_scaled)
rf_mse = mean_squared_error(y_test, rf_pred)
rf_r2 = r2_score(y_test, rf_pred)

print("Random Forest Results")
print("MSE:", rf_mse)
print("R² Score:", rf_r2)


Random Forest Results
MSE: 122017969.83678594
R² Score: 0.8045981720315679


# KNN 5

In [167]:
# Constructing training model
n_neighbors=5
knn5=KNeighborsRegressor(n_neighbors,weights='uniform')
knn5.fit(X_train_scaled,y_train)
y1_knn5=knn5.predict(X_train_scaled)
y1_knn5=list(y1_knn5)

In [168]:
# Evaluating model on training data
mse = mean_squared_error(y_train, y1_knn5)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn5.score(X_train_scaled,y_train)}")

The RMSE is: 11189.617440356862
The R-squared is 0.7431721574962407


In [169]:
# Using model on test data
y2_knn5=knn5.predict(X_test_scaled)
y2_knn5=list(y2_knn5)

In [170]:
# Evaluating model on test data
mse = mean_squared_error(y_test, y2_knn5)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn5.score(X_test_scaled,y_test)}")

The RMSE is: 16419.9884314175
The R-squared is 0.5682319313686602


# KNN 2

In [171]:
# Constructing model
n_neighbors=2
knn2=KNeighborsRegressor(n_neighbors,weights='uniform')
knn2.fit(X_train_scaled,y_train)
y1_knn2=knn2.predict(X_train_scaled)
y1_knn2=list(y1_knn2)

In [172]:
#Training
mse = mean_squared_error(y_train, y1_knn2)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn2.score(X_train_scaled,y_train)}")

The RMSE is: 8869.58205730527
The R-squared is 0.8386318062358111


In [173]:
# Testing
y2_knn2=knn2.predict(X_test_scaled)
y2_knn2=list(y2_knn2)

In [174]:
mse = mean_squared_error(y_test, y2_knn2)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn2.score(X_test_scaled,y_test)}")

The RMSE is: 16570.30287065873
The R-squared is 0.5602906299791762


# KNN 12

In [175]:
# Constructing Model
n_neighbors=12
knn12=KNeighborsRegressor(n_neighbors,weights='uniform')
knn12.fit(X_train_scaled,y_train)
y1_knn12=knn12.predict(X_train_scaled)
y1_knn12=list(y1_knn12)

In [176]:
# Training
mse = mean_squared_error(y_train, y1_knn12)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn12.score(X_train_scaled,y_train)}")

The RMSE is: 12382.265467316043
The R-squared is 0.6855063742706389


In [177]:
# Testing
y2_knn12=knn12.predict(X_test_scaled)
y2_knn12=list(y2_knn12)
mse = mean_squared_error(y_test, y2_knn12)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {knn12.score(X_test_scaled,y_test)}")

The RMSE is: 15441.599353734106
The R-squared is 0.6181529958901539


# SVC - Linear

In [178]:
# Constructing the model
from sklearn import svm
vect = svm.SVC(kernel='linear')
vect.fit(X_train_scaled,y_train)
y1_svc=vect.predict(X_train_scaled)
y1_svc=list(y1_svc)

In [179]:
# Training
mse = mean_squared_error(y_train, y1_svc)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {vect.score(X_train_scaled,y_train)}")

The RMSE is: 9313.070439157516
The R-squared is 0.3565149136577708


In [180]:
# Testing
y2_svc=vect.predict(X_test_scaled)
y2_svc=list(y2_svc)
mse = mean_squared_error(y_test, y2_svc)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {vect.score(X_test_scaled,y_test)}")

The RMSE is: 16048.664614449812
The R-squared is 0.27840552416823605


# SVC - RBF

In [181]:
# Constructing model
rbf_svc =svm.SVC(kernel='rbf')
rbf_svc.fit(X_train_scaled,y_train)


SVC()

In [182]:
# Training
y1_svc=rbf_svc.predict(X_train_scaled)
y1_svc=list(y1_svc)
mse = mean_squared_error(y_train, y1_svc)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {rbf_svc.score(X_train_scaled,y_train)}")


The RMSE is: 20564.879910038027
The R-squared is 0.3770015698587127


In [183]:
# Testing
y2_svc=rbf_svc.predict(X_test_scaled)
y2_svc=list(y2_svc)
mse = mean_squared_error(y_test, y2_svc)
rmse = np.sqrt(mse)

print(f"The RMSE is: {rmse}")
print(f"The R-squared is {rbf_svc.score(X_test_scaled,y_test)}")

The RMSE is: 24479.073656512806
The R-squared is 0.2859384808537351


# XGBoost

In [184]:
from xgboost import XGBRegressor
# Initialize and train XGBoost
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train_scaled, y_train)

# Predict and evaluate
xgb_pred = xgb_model.predict(X_test_scaled)
xgb_mse = mean_squared_error(y_test, xgb_pred)
xgb_r2 = r2_score(y_test, xgb_pred)

print("XGBoost Results")
print("MSE:", xgb_mse)
print("R² Score:", xgb_r2)

XGBoost Results
MSE: 118207295.67603499
R² Score: 0.810700655934544


# Neural Network

In [185]:
# Define a simple Neural Network
nn_model = Sequential([
    Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile the model
nn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history = nn_model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_train_scaled, y_test), verbose=1)

# Predict and evaluate
nn_pred = nn_model.predict(X_test_scaled).flatten()
nn_mse = mean_squared_error(y_test, nn_pred)
nn_r2 = r2_score(y_test, nn_pred)

print("Neural Network Results")
print("MSE:", nn_mse)
print("R² Score:", nn_r2)

NameError: name 'Sequential' is not defined